##50k reviews

In [116]:
%tensorflow_version 1.x

In [117]:
from keras.datasets import imdb

#vectorized data
vocabulary_size = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocabulary_size)
print('Loaded dataset with {} training samples, {} test samples'.format(len(X_train), len(X_test)))
print('---review---')
print(X_train[6])
print('---label---')
print(y_train[6])

Loaded dataset with 25000 training samples, 25000 test samples
---review---
[1, 2, 365, 1234, 5, 1156, 354, 11, 14, 2, 2, 7, 1016, 2, 2, 356, 44, 4, 1349, 500, 746, 5, 200, 4, 4132, 11, 2, 2, 1117, 1831, 2, 5, 4831, 26, 6, 2, 4183, 17, 369, 37, 215, 1345, 143, 2, 5, 1838, 8, 1974, 15, 36, 119, 257, 85, 52, 486, 9, 6, 2, 2, 63, 271, 6, 196, 96, 949, 4121, 4, 2, 7, 4, 2212, 2436, 819, 63, 47, 77, 2, 180, 6, 227, 11, 94, 2494, 2, 13, 423, 4, 168, 7, 4, 22, 5, 89, 665, 71, 270, 56, 5, 13, 197, 12, 161, 2, 99, 76, 23, 2, 7, 419, 665, 40, 91, 85, 108, 7, 4, 2084, 5, 4773, 81, 55, 52, 1901]
---label---
1


In [118]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##2k reviews

In [119]:
import pandas as pd
total = pd.read_csv('/content/drive/MyDrive/Movie_Reviews/movie_review.csv')
total 

#group by html_id
total = total.groupby('html_id').agg(lambda x: x.tolist())
total = total.reset_index()
total

#drop other columns: only conserve text
total.columns
total = total.drop(['html_id', 'fold_id', 'cv_tag', 'sent_id'], axis=1)
total

,text,tag
0,"[warning : anyone offended by blatant , leerin...","[pos, pos, pos, pos, pos, pos, pos, pos, pos, ..."
1,[various films seen at the seattle film festiv...,"[neg, neg, neg, neg, neg, neg, neg, neg, neg, ..."
2,"[every year--every year at the festival , i wa...","[pos, pos, pos, pos, pos, pos, pos, pos, pos, ..."
3,[this is a film that i was inclined to like at...,"[neg, neg, neg, neg, neg, neg, neg, neg, neg, ..."
4,[i'm not sure i should be writing a review of ...,"[neg, neg, neg, neg, neg, neg, neg, neg, neg, ..."
...,...,...
1995,[one might expect a cathartic viewing experien...,"[neg, neg, neg, neg, neg, neg, neg, neg, neg, ..."
1996,"[susan granger's review of "" mulholland drive ...","[neg, neg, neg, neg, neg, neg, neg, neg, neg, ..."
1997,[historical epic as a genre was almost banishe...,"[pos, pos, pos, pos, pos, pos, pos, pos, pos, ..."
1998,"[all great things come to an end , and the dot...","[pos, pos, pos, pos, pos, pos, pos, pos, pos, ..."


In [120]:
#the result is a separated chunks for every reviews: 
#chunk1, chunk2, ..., sentiment
total.values

array([[list(['warning : anyone offended by blatant , leering machismo had better avoid this film .', 'or lots of blood & guts , men against men and mano-et-mano stuff .', "in other words , it's a walter hill film !", 'with a john milius script !', 'i always picture these guys getting together and producing a movie between arm-wrestling matches .', 'these films always contain male characters i have a very hard time identifying with , probably due to the likelihood that any meeting between them and me would result in my arm being ripped off and then my subsequent death by beating with said limb .', "and we got tough guys galore , here ; drug-running banditos by the dozens , all dirty and sweaty and pretty ill-tempered , overall ; a secret task force of army commandos who are in the area to cover-up ( supposedly ) any connection between the government and the drug runners ; and lots of shit-kicking texas dirt farmers who'd as soon shoot you between the eyes as look at you .", 'in particu

In [121]:
#we merge the chunks together and we obtain: 
#review, sentiment
total_text = list()
for lists in total.values:
  combines_text = ''
  for _ in lists[0]:
    combines_text = combines_text + _
  total_text.append([combines_text, lists[1][0]])

#total_text
total_text = pd.DataFrame(total_text)
total_text

,0,1
0,"warning : anyone offended by blatant , leering...",pos
1,various films seen at the seattle film festiva...,neg
2,"every year--every year at the festival , i wai...",pos
3,this is a film that i was inclined to like at ...,neg
4,i'm not sure i should be writing a review of t...,neg
...,...,...
1995,one might expect a cathartic viewing experienc...,neg
1996,"susan granger's review of "" mulholland drive ""...",neg
1997,historical epic as a genre was almost banished...,pos
1998,"all great things come to an end , and the dot-...",pos


In [122]:
X_train = total_text[0][0:1000].tolist()
y_train_ = total_text[1][0:1000].tolist()
X_test = total_text[0][1000:2000].tolist()
y_test_ = total_text[1][1000:2000].tolist()

y_train = list()
y_test = list()

#labeling positive and negative to 0
for _ in y_train_:
  if _ == 'pos':
    y_train.append(0)
  else:
    y_train.append(1)

for _ in y_test_:
  if _ == 'pos':
    y_test.append(0)
  else:
    y_test.append(1)

print(len(X_train), len(y_test), len(X_test), len(y_test))

X_train_ = X_train.copy()
X_test_ = X_test.copy()
X_train = list() # to dust return
X_test = list()
print(len(X_train_), len(y_test), len(X_test_), len(y_test))

#tokenization of reviews 
import re
print(X_train_[0])
print(X_train_[1])

for _ in X_train_:
  X_train.append(re.sub("[^\w]", " ",  _).split())

print(X_test_[0])
print(X_test_[1])
for _ in X_test_:
  X_test.append(re.sub("[^\w]", " ",  _).split())

print(len(X_train), len(y_test), len(X_test), len(y_test))

1000 1000 1000 1000
1000 1000 1000 1000
various films seen at the seattle film festival :it's all true/three men on a raft ( u . s . a . , 1942/1986 ) director : orson wellesa recently uncovered one-fourth of orson welle's planned " docu-drama " of amazing stories around the world named it's all true , this short dealt with welle's attempts to document the actions of several brazilian fisherman in 1942 who had become folk heroes for their fantastic voyage to rio on a raft .while several clips of the film are seen , the short tends to spend more time trying to recount welle's attempts to film it , despite studio problems , a staggering workload , and extremely ill fortune on the set .impressive photography , but really only interesting to film historians .sadie thompson ( u . s . a . , 1928 ) director : raoul walsh screenwriter : raoul walsh , from the john colton play " rain " , based on w . somerset maugham's story " miss thompson " cast : gloria swanson , raoul walsh , lionel barrymo

In [123]:
#encoding: words to numbers
#index based encoding
from keras.preprocessing.text import Tokenizer

t = Tokenizer()
print(X_train[0])
#fit the tokenizer on the documents
t.fit_on_texts(X_train + X_test)
print(X_train[0])
#summarize what was learned
print(t.word_counts)
print(t.document_count)
print(t.word_index)
print(t.word_docs)

#create sequences of integers
X_train = t.texts_to_sequences(X_train)
X_test = t.texts_to_sequences(X_test)
print(X_train[0])

['warning', 'anyone', 'offended', 'by', 'blatant', 'leering', 'machismo', 'had', 'better', 'avoid', 'this', 'film', 'or', 'lots', 'of', 'blood', 'guts', 'men', 'against', 'men', 'and', 'mano', 'et', 'mano', 'stuff', 'in', 'other', 'words', 'it', 's', 'a', 'walter', 'hill', 'film', 'with', 'a', 'john', 'milius', 'script', 'i', 'always', 'picture', 'these', 'guys', 'getting', 'together', 'and', 'producing', 'a', 'movie', 'between', 'arm', 'wrestling', 'matches', 'these', 'films', 'always', 'contain', 'male', 'characters', 'i', 'have', 'a', 'very', 'hard', 'time', 'identifying', 'with', 'probably', 'due', 'to', 'the', 'likelihood', 'that', 'any', 'meeting', 'between', 'them', 'and', 'me', 'would', 'result', 'in', 'my', 'arm', 'being', 'ripped', 'off', 'and', 'then', 'my', 'subsequent', 'death', 'by', 'beating', 'with', 'said', 'limb', 'and', 'we', 'got', 'tough', 'guys', 'galore', 'here', 'drug', 'running', 'banditos', 'by', 'the', 'dozens', 'all', 'dirty', 'and', 'sweaty', 'and', 'pretty

In [124]:
#number of labels
num_words = len(t.word_index) + 1
num_words

40181

In [125]:
#pad sequences of integers
from keras.preprocessing import sequence
max_words = 2462
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [126]:
#max number of characters in a review
max1 = 0
for _ in (X_train + X_test):
  if max(_) > max1:
    max1 = max(_)
max1

73281

In [137]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten
embedding_size=32
model=Sequential()
model.add(Embedding(40181, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dropout(60))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 2462, 32)          1285792   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 2462, 32)          0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 2462, 250)         283000    
_________________________________________________________________
lstm_11 (LSTM)               (None, 100)               140400    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 1,709,293
Trainable params: 1,709,293
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
batch_size = 64
num_epochs = 5
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train_, y_train_ = X_train[batch_size:], y_train[batch_size:]
model.fit(X_train_, y_train_, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)
#model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, epochs=num_epochs)

In [129]:
scores = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.7590000033378601


In [130]:
print(X_test[0])

[   0    0    0 ... 1144    7 3813]


In [131]:
import pickle
f = open('my_lstm.pickle', 'wb')
pickle.dump(model, f)
f.close()

In [132]:
from keras.preprocessing.text import Tokenizer

tok = Tokenizer()  

testing_list = list()
testing= "warning : anyone offended by blatant , leering machismo had better avoid this film .or lots of blood & guts , men against men and mano-et-mano stuff .in other words , it's a walter hill film !with a john milius script !i always picture these guys getting together and producing a movie between arm-wrestling matches .these films always contain male characters i have a very hard time identifying with , probably due to the likelihood that any meeting between them and me would result in my arm being ripped off and then my subsequent death by beating with said limb .and we got tough guys galore , here ; drug-running banditos by the dozens , all dirty and sweaty and pretty ill-tempered , overall ; a secret task force of army commandos who are in the area to cover-up ( supposedly ) any connection between the government and the drug runners ; and lots of shit-kicking texas dirt farmers who'd as soon shoot you between the eyes as look at you .in particular , we got nick nolte as one hard-ass texas ranger , powers boothe as the drug kingpin , michael ironsides as the leader of the secret army , and rip torn as the local sheriff .torn is the sympathetic figure of the group ; he smiles before shooting anyone .as to women . . .well , i've never seen jane fonda or meryl streep in a walter hill film , and at this rate , i doubt i ever will .women exist here to look good , comfort the man , and get argued over .gosh !just like the old days . . .frankly , this is a pretty good movie , if you can accept the premise and can take the macho stuff .the cinematography is excellent , the cast of characters is broad and has texture , the script is quite good , and the film lets you keep up with what's happening yourself , without spelling it out to you .i appreciate a film that makes me have to think to keep up .finally , there's lots of sam peckinpah slow-motion shoot-ups ."
testing2 = "my final criticism of the film lies in its glamorous depiction of alcohol and alcoholism "
testing3 = "this film brims with imagination , containing a lush imagery that shows the arcadia that is heaven and the bitterness and frightfulness that is hell"
testing_list.append(re.sub("[^\w]", " ",  testing2).split())

tok.fit_on_texts(testing_list)

testing_list = tok.texts_to_sequences(testing_list)  

print(tok.word_counts)
print(tok.document_count)
print(tok.word_index)
print(tok.word_docs)
print(testing_list[0])
#pad sequences of integers
from keras.preprocessing import sequence
max_words = 2462
testing_list = sequence.pad_sequences(testing_list, maxlen=max_words)

model.predict(testing_list)

OrderedDict([('my', 1), ('final', 1), ('criticism', 1), ('of', 2), ('the', 1), ('film', 1), ('lies', 1), ('in', 1), ('its', 1), ('glamorous', 1), ('depiction', 1), ('alcohol', 1), ('and', 1), ('alcoholism', 1)])
1
{'of': 1, 'my': 2, 'final': 3, 'criticism': 4, 'the': 5, 'film': 6, 'lies': 7, 'in': 8, 'its': 9, 'glamorous': 10, 'depiction': 11, 'alcohol': 12, 'and': 13, 'alcoholism': 14}
defaultdict(<class 'int'>, {'of': 1, 'glamorous': 1, 'depiction': 1, 'final': 1, 'criticism': 1, 'the': 1, 'and': 1, 'my': 1, 'its': 1, 'in': 1, 'alcoholism': 1, 'lies': 1, 'film': 1, 'alcohol': 1})
[2, 3, 4, 1, 5, 6, 7, 8, 9, 10, 11, 1, 12, 13, 14]


array([[0.9984591]], dtype=float32)